<a href="https://colab.research.google.com/github/MdKaif2782/kuet-hackathon-preli/blob/main/kuet_preli_prblem_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets==2.14.5
!pip install transformers==4.28.0
!pip install protobuf==3.20.*

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2023.6.0 which is incompatible.
     ━━━━━━

In [2]:
from datasets import load_dataset

dataset = load_dataset("SKNahin/bengali-transliteration-data")

split_dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)

train_dataset = split_dataset['train']
val_dataset = split_dataset['test']

print(f"Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/5006 [00:00<?, ? examples/s]

Training samples: 4004, Validation samples: 1002


In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, Trainer, TrainingArguments
import torch  # Ensure torch is imported

# Load mBART model and tokenizer
model_name = "facebook/mbart-large-50"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

# Set tokenizer's source and target language
tokenizer.src_lang = "en_XX"
tokenizer.tgt_lang = "bn_IN"

# Preprocessing function for mBART
def preprocess(batch):
    inputs = tokenizer(batch["rm"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(batch["bn"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply preprocessing
train_dataset = train_dataset.map(preprocess, batched=True)
val_dataset = val_dataset.map(preprocess, batched=True)

# Set dataset format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Define training arguments
training_args = TrainingArguments(
    output_dir="./mbart_results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./mbart_logs",
    logging_steps=10,
    save_steps=500,
    fp16=torch.cuda.is_available(),  # Use mixed precision if GPU is available
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

...

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a MBart50TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2664: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  else torch.cuda.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)


Epoch,Training Loss,Validation Loss
1,0.115400,0.101141
2,0.043900,0.082427


/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2664: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  else torch.cuda.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2664: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  else torch.cuda.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2664: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  else torch.cuda.amp.autocast(cache_enabled=cache_enabled, dtype=self.amp_dtype)
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2664: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', a

In [ ]:
import torch

sample = val_dataset.select(range(10))
inputs = sample["input_ids"]

if torch.cuda.is_available():
    inputs = inputs.cuda()

preds = model.generate(inputs)

decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True, clean_up_tokenization_spaces=True) for pred in preds]
decoded_labels = [tokenizer.decode(label, skip_special_tokens=True, clean_up_tokenization_spaces=True) for label in sample["labels"]]

for i, (pred, label) in enumerate(zip(decoded_preds, decoded_labels)):
    print(f"Sample {i + 1}")
    print(f"Prediction: {pred}")
    print(f"Label: {label}\n")


Sample 1
Prediction: <extra_id_0>.ননন  আ।  আ। ।।।।।।।।
Label: ভালো করে ট্রাই করেন পাবেন..

Sample 2
Prediction: <extra_id_0> তে                 
Label: এতো বানান ভুল করছেন বারে

Sample 3
Prediction: <extra_id_0>?নননননননননননননননননন
Label: যা শুরু করছেন ভাই?

Sample 4
Prediction: <extra_id_0>. পোনন  আ।  । ।।। । । 
Label: মাশাল্লাহ অনেক সুন্দর পোষ্ট।

Sample 5
Prediction: <extra_id_0> তে
Label: এতো কিছুর দরকার হয় না... জাস্ট মডিউল ফ্ল্যাশ ইজ এনাফ... আর এই সিস্টেম এ অন্য ইমজি মারলে ফেইসবুক, মেসেঞ্জার ওদের নিজেদের ইমজিই শো করবে

Sample 6
Prediction: <extra_id_0>.ননননন ।   ন       
Label: শাওমি রেডমি ৬এ রয় হয়না ভাই

Sample 7
Prediction: <extra_id_0>.ননন নন  ননন       
Label: জি ইনশাল্লাহ।

Sample 8
Prediction: <extra_id_0> পোন পো      আ          
Label: আজাইরা পোস্ট আরে হ্যাক হই

Sample 9
Prediction: <extra_id_0> তে                 
Label: এখন তো স্নাপটিউব থেকে ডাউনলোড করা যায় না

Sample 10
Prediction: <extra_id_0> তে     আ            
Label: আপনি ট্রাই করে দাকসান



In [ ]:
model.save_pretrained("./banglish-to-bangla")
tokenizer.save_pretrained("./banglish-to-bangla")

('./banglish-to-bangla/tokenizer_config.json',
 './banglish-to-bangla/special_tokens_map.json',
 './banglish-to-bangla/spiece.model',
 './banglish-to-bangla/added_tokens.json',
 './banglish-to-bangla/tokenizer.json')

In [ ]:
import torch

def translate_banglish_to_bangla(model, tokenizer, banglish_input):
    inputs = tokenizer(banglish_input, return_tensors="pt", padding=True, truncation=True, max_length=128)

    if torch.cuda.is_available():
        inputs = {key: value.cuda() for key, value in inputs.items()}
        model = model.cuda()

    translated_tokens = model.generate(**inputs, decoder_start_token_id=tokenizer.lang_code_to_id["bn_IN"])
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

    return translated_text

# Take custom input
print("Enter your Banglish text (type 'exit' to quit):")
while True:
    banglish_text = input("Banglish: ")
    if banglish_text.lower() == "exit":
        break

    # Translate Banglish to Bangla
    translated_text = translate_banglish_to_bangla(model, tokenizer, banglish_text)
    print(f"Translated Bangla: {translated_text}\n")


Enter your Banglish text (type 'exit' to quit):
Banglish: Ekekta telbaj salara
Translated Bangla: <extra_id_0>.নন ন ন ন ন ন  ।   

Banglish: eto koster ki ache shakibwap.tk,shakibwap.ml&aiovideodl.ml theke shohojei downlod kora jay
Translated Bangla: <extra_id_0>.টান                

Banglish: amar
Translated Bangla: <extra_id_0> করন করন  ।নননননননননননন

Banglish: dhon
Translated Bangla: <extra_id_0>.ন ননননননননন নন নন 



KeyboardInterrupt: Interrupted by user

In [ ]:
# prompt: download the banglish-to-bangla folder as zip

from google.colab import files
import zipfile

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

import os
if not os.path.exists("./banglish-to-bangla"):
    print("Directory ./banglish-to-bangla not found. Please run the training code first.")
else:
  zipf = zipfile.ZipFile('banglish-to-bangla.zip', 'w', zipfile.ZIP_DEFLATED)
  zipdir('./banglish-to-bangla', zipf)
  zipf.close()
  files.download('banglish-to-bangla.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>